In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from config import CLIENT_ID, REDIRECT_URI,CLIENT_SECRET
import tkinter as tk


# Set up Spotify API authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope='playlist-modify-public user-library-read playlist-read-private'))

In [34]:

def get_liked_songs():
    # Get the user's liked songs
    results = sp.current_user_saved_tracks()
    liked_songs = [(item['track']['name'], item['track']['id']) for item in results['items']]
    return liked_songs


def search_and_create_playlist(playlist_names, liked_songs, name_of_playlist):
    new_playlist = sp.user_playlist_create(user=sp.me()['id'], name=name_of_playlist)
    added_song_ids = set()

    for playlist_name in playlist_names:
        # Search for public playlists containing the specified name
        playlists = sp.search(q=playlist_name, type='playlist')
        if playlists['playlists']['total'] > 0:
            for playlist in playlists['playlists']['items']:
                playlist_id = playlist['id']
                playlist_tracks = sp.playlist_tracks(playlist_id)
                
                # Check if playlist_tracks is not None before accessing its items
                if playlist_tracks is not None and 'items' in playlist_tracks:
                    try:
                        track_ids = [item['track']['id'] for item in playlist_tracks['items']]
                        
                        for song_name, song_id in liked_songs:
                            if song_id in track_ids and song_id not in added_song_ids:
                                sp.playlist_add_items(new_playlist['id'], [song_id])
                                added_song_ids.add(song_id)
                    except:
                        pass
                else:
                    print(f"Skipping playlist '{playlist_name}' due to missing or empty tracks.")
        else:
            print(f"No playlists found containing the name '{playlist_name}'.")


def get_user_choice(name_of_playlist):
    root = tk.Tk()
    root.title("Playlist Choice")

    user_choice = tk.StringVar()

    def on_button_click(choice):
        user_choice.set(choice)
        root.destroy()

    tk.Label(root, text="Select Playlist Type:").pack()
    i = 0
    for name in name_of_playlist:
        tk.Button(root, text=name, command=lambda: on_button_click(i)).pack()
        i+1

    root.mainloop()

    return user_choice.get()


if __name__ == "__main__":
    name_of_playlist = ['Gym_Playlist','Happy_Playlist','Classic Rock']
    choice = int(get_user_choice(name_of_playlist))
    liked_songs = get_liked_songs()
    
    Gym = ['WORKOUT', 'GYM', 'LIFT']
    Happy = ['HAPPY', 'Vibes', 'Uplifting']
    Classic_rock = ['Rock And Roll', 'Dad Rock', '80s Music']
    playlist_names_to_search = [Gym,Happy,Classic_rock]
    
    search_and_create_playlist(playlist_names_to_search[choice], liked_songs, name_of_playlist[choice])